In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9052,2024-06-14,Bolívia Libobasquet,21:30,Leones de Potosi,Kinwa,1.07,7.17,163.5,1.81,1.85,0.0,0.00,0.00,h8Ee51GL,0.934579,0.139470,0.552486,0.540541,0.074049,119.942,26.423573,0.220303,1.8,1.643168,0.912871,91.16,1.380,0.311047,0.225396,57.0,354.108,179.963657,0.508217,0.6,1.341641,2.236068,430.68,4.392,2.425215,0.552189,-32.0,86,74,1.06,5.82,122.136,293.034,1.046930,0.015456,NaN,-1.63,-0.326,-0.214724,0.000000,0.0,0.000000,-3.44,-0.688,-8.968023,0.000000,0.0,0.000000
9053,2024-06-14,Chile Lnb,21:30,Leones Quilpue,U. De Concepcion,1.47,2.56,158.5,1.85,1.85,0.0,0.00,0.00,OCQsULTj,0.680272,0.390625,0.540541,0.540541,0.070897,398.888,678.631638,1.701309,2.4,1.341641,0.559017,81.75,6.072,11.140118,1.834670,66.0,153.928,93.632890,0.608290,2.4,1.341641,0.559017,118.11,1.780,1.124455,0.631717,29.0,75,93,1.09,1.27,105.276,148.726,0.382504,0.000000,NaN,0.64,0.128,3.671875,0.875923,1.0,0.124077,2.22,0.444,3.513514,0.766956,0.8,0.033044
9054,2024-06-14,Colômbia Lbp,21:40,Caribbean Storm Islands,Bucaros,1.31,3.25,158.5,1.85,1.85,0.0,0.00,0.00,d64JucFD,0.763359,0.307692,0.540541,0.540541,0.071051,176.100,143.598065,0.815435,1.8,1.643168,0.912871,133.48,2.350,2.130411,0.906558,25.0,120.120,11.996239,0.099869,3.0,0.000000,0.000000,117.18,1.344,0.199073,0.148120,47.0,94,93,1.42,1.26,128.356,0.000,0.601661,0.000000,NaN,0.76,0.152,2.039474,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9055,2024-06-14,Eua Wnba,20:30,Minnesota Lynx F,Los Angeles Sparks F,1.09,7.90,159.5,1.86,1.92,0.0,0.00,0.00,hvwJIJ6o,0.917431,0.126582,0.537634,0.520833,0.044013,178.352,95.107536,0.533257,1.8,1.643168,0.912871,338.00,2.088,0.991146,0.474687,34.0,222.674,105.367656,0.473192,0.6,1.341641,2.236068,410.80,2.898,1.311705,0.452624,-37.0,100,79,3.38,5.20,188.196,0.000,1.071279,0.022448,NaN,3.20,0.640,0.140625,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9056,2024-06-14,Eua Wnba,20:30,Washington Mystics F,Chicago Sky F,2.15,1.71,156.5,1.84,1.92,1.5,2.00,1.56,WUyNHwMi,0.465116,0.584795,0.543478,0.520833,0.049912,311.052,219.189284,0.704671,0.0,0.000000,NaN,147.74,4.356,2.823053,0.648084,-44.0,178.350,68.912970,0.386392,0.6,1.341641,2.236068,243.00,2.356,0.921591,0.391168,-23.0,83,75,1.78,3.24,0.000,338.718,0.161206,0.030090,0.174790,0.00,0.000,inf,0.000000,0.0,0.000000,8.77,1.754,0.404789,0.000000,0.0,0.000000
9057,2024-06-14,Filipinas Copa Das Filipinas,08:30,San Miguel Beermen,Meralco Bolts,1.66,2.19,190.5,1.86,1.80,0.0,0.00,0.00,6Jzn9Ufe,0.602410,0.456621,0.537634,0.555556,0.059031,154.436,18.681211,0.120964,3.0,0.000000,0.000000,179.82,1.452,0.096281,0.066309,31.0,213.964,41.147054,0.192308,1.2,1.643168,1.369306,230.28,2.406,0.311897,0.129633,-2.0,111,101,1.62,2.28,136.894,185.722,0.194684,0.023184,NaN,-0.03,-0.006,-110.000000,0.000000,0.0,0.000000,3.52,0.704,1.690341,0.000000,0.0,0.000000
9058,2024-06-14,Grécia Liga De Basquete,15:15,Panathinaikos,Olympiakos,1.38,3.08,156.5,1.87,1.79,0.0,0.00,0.00,0j7iAH4k,0.724638,0.324675,0.534759,0.558659,0.049313,142.242,56.131078,0.394617,2.4,1.341641,0.559017,203.28,1.662,0.624476,0.375738,37.0,143.392,55.526031,0.387232,1.8,1.643168,0.912871,137.70,1.734,0.784207,0.452253,20.0,88,85,2.31,1.62,100.964,148.368,0.539050,0.030912,NaN,0.76,0.152,2.500000,0.858569,0.9,0.041431,-1.48,-0.296,-7.0270

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9056,20:30,Eua Wnba,Washington Mystics F,Chicago Sky F,2.15,Back Home
9059,21:15,Uruguai Liga Uruguaia,Aguada,Penarol,1.67,Back Home
9066,21:00,Porto Rico Bsn,Mets de Guaynabo,Gigantes de Carolina,1.75,Back Home
9073,21:00,Brasil Lbf Feminina,Corinthians F,Ituano F,2.76,Back Home
